In [1]:
import os
import warnings
import openpyxl
from groq import Groq
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font
import os
from datetime import datetime
from tkinter import filedialog
import tkinter as tk
import requests
from bs4 import BeautifulSoup

# Suppress warnings
warnings.filterwarnings('ignore')

# Set the API Key as an Environment Variable
os.environ['GROQ_API_KEY'] = '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'

# Set USER_AGENT environment variable
os.environ['USER_AGENT'] = 'GMA_News_AI_Research_Tool/1.0'

# Chatbot Setup
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# File Paths
file_path = r"C:\Users\104288\UL Solutions\GMA - Global Market Access - AI POC\GMA News\Input\GMA_News_URL_Input_CENELEC.xlsx"

# Load the workbook and select the sheets
wb = openpyxl.load_workbook(file_path, data_only=True)
output_sheet = wb['Output']

# List of attributes
attributes = [
    "Title", "Summary", "Affected Products", "Summary Details", "Conformity Assessment",
    "Marking Requirement", "Technical Requirement", "Publish Date", "Effective Date",
    "Mandatory Date", "Consultation Closing"
]

def display_topic_info(date, topic, url):
    info = f'<b>Processing Topic:</b> {topic}<br>'
    if date:
        info += f'<b>Date:</b> {date}<br>'
    if url:
        info += f'<b>URL:</b> {url}<br>'
    display(HTML(info + '<br>'))

def get_text_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()


def get_text_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()

def process_url(date, topic, url, attribute_outputs):
    try:
        date_str = date.strftime('%Y-%m-%d') if isinstance(date, datetime) else str(date)
        output_path = rf"C:\Users\104288\UL Solutions\GMA - Global Market Access - AI POC\GMA News\Bulk_RAG\Output\{date_str}_{topic}_Output.xlsx"
        
        # 显示处理信息
        display(HTML(f"""
        <div style='background-color: #f0f0f0; padding: 10px; margin: 10px 0;'>
            <h3>正在处理:</h3>
            <p><b>主题:</b> {topic}</p>
            <p><b>日期:</b> {date_str}</p>
            <p><b>URL:</b> {url}</p>
        </div>
        """))
        
        # 获取URL内容
        text_content = get_text_content(url)
        if not text_content:
            raise ValueError("无法获取URL内容")
            
        # 处理每个属性
        for row_index, output_row in enumerate(output_sheet.iter_rows(min_row=2, values_only=True), start=2):
            if not output_row or len(output_row) < 4:
                continue
                
            try:
                attribute, prompt = output_row[0], output_row[1]
                if not attribute or not prompt:
                    continue
                    
                # 构建提示词
                formatted_prompt = (
                    f"As a product regulatory compliance officer, analyze the following text content about {topic}, published on {date}. "
                    f"Focus on the attribute: '{attribute}'. "
                    f"Specific Question: {prompt} "
                    "Answer the specific question based on the provided text content, but do not repeat the question in the answer. "
                    "Be very concise and give answer directly. When giving a negative answer, simply state 'Not Need'. "
                    "Avoid repeating the question or using multiple sentences to say the same thing. "
                    "Only use information from the provided text content. "
                    "If your answer contains multiple points, please use bullet points and ensure each point is on a new line."
                    "If your answer contains date, please just output the date without description."
                )
                
                # 更新提示词显示
                if attribute in attribute_outputs:
                    attribute_outputs[attribute]['prompt'].value = formatted_prompt
                
                # 获取AI响应
                full_prompt = formatted_prompt + f"\nText content: {text_content[:20000]}... "
                chat_completion = client.chat.completions.create(
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that refines and improves answers."},
                        {"role": "user", "content": full_prompt}
                    ],
                    model="llama3-8b-8192",
                )
                
                final_answer = chat_completion.choices[0].message.content.strip()
                
                # 更新AI输出显示
                if attribute in attribute_outputs:
                    attribute_outputs[attribute]['ai_output'].value = final_answer
                
                # 显示处理结果
                display(HTML(f"""
                <div style='border: 1px solid #ddd; padding: 10px; margin: 10px 0;'>
                    <h4>属性: {attribute}</h4>
                    <div style='margin: 10px 0;'>
                        <b>提示词:</b><br>
                        <pre style='background-color: #f8f8f8; padding: 10px;'>{formatted_prompt}</pre>
                    </div>
                    <div style='margin: 10px 0;'>
                        <b>AI响应:</b><br>
                        <pre style='background-color: #f8f8f8; padding: 10px;'>{final_answer}</pre>
                    </div>
                </div>
                """))
                
            except Exception as row_error:
                display(HTML(f"<div style='color: red;'><b>处理行 {row_index} 时出错:</b> {str(row_error)}</div>"))
        
        # 保存结果
        wb.save(output_path)
        display(HTML(f"<div style='color: green;'><b>结果已保存到:</b> {output_path}</div>"))
        
    except Exception as e:
        display(HTML(f"<div style='color: red;'><b>处理URL时出错:</b> {str(e)}</div>"))
# Custom CSS for improved styling
custom_css = """
<style>
.widget-label {
    font-weight: bold;
    font-size: 16px;
}
.widget-text, .widget-textarea {
    border: 1px solid #ddd;
    border-radius: 4px;
    font-size: 16px;
    color: #333;
}
.widget-button {
    background-color: #b30000;
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 4px;
    font-size: 16px;
    font-weight: bold;
    text-align: center;
    width: 150px !important;
    height: 40px !important;
}
.widget-button:hover {
    background-color: #cc0000;
}
.output_area {
    border: 1px solid #ddd;
    border-radius: 4px;
    padding: 10px;
    margin-top: 10px;
    font-size: 16px;
    color: #333;
}
.title {
    font-size: 30px;
    color: #b30000;
    text-align: left;
    font-weight: bold;
    margin-bottom: 30px;
}
.note {
    font-size: 14px;
    color: #666;
    font-style: italic;
    margin-top: 10px;
}
.jupyter-widgets-output-area .p-TabBar-tabLabel {
    white-space: normal;
    word-wrap: break-word;
    max-width: 200px;
    line-height: 1.2;
    padding: 8px 4px;
}
.jupyter-widgets-output-area .p-TabBar-tab {
    min-height: 50px;
    max-height: 60px;
}

.p-TabBar-tabLabel {
    white-space: normal !important;
    word-wrap: break-word !important;
    max-width: none !important;
    line-height: 1.2;
    padding: 5px 10px;
    font-size: 14px;
    height: 40px !important;        /* 固定高度 */
    display: flex !important;       /* 使用flex布局 */
    align-items: center !important; /* 垂直居中 */
    justify-content: center !important; /* 水平居中 */
}

.p-TabBar-tab {
    height: 50px !important;        /* 固定标签高度 */
    max-width: none !important;
    display: flex !important;
    align-items: center !important;
}

.jupyter-widgets-output-area .p-TabBar {
    height: 50px !important;        /* 固定标签栏高度 */
    display: flex !important;
    align-items: center !important;
}

.p-TabPanel {
    padding: 15px;
}

/* 修改滑块轨道样式 */
.widget-slider .ui-slider {
    height: 10px !important;              /* 增加滑块轨道高度 */
    background: #e6e6e6 !important;       /* 轨道背景色 */
    border: none !important;
    border-radius: 5px !important;
}

/* 修改滑块填充部分样式 */
.widget-slider .ui-slider-range {
    height: 10px !important;              /* 增加填充区域高度 */
    background: #2185d0 !important;       /* 蓝色填充 */
    border-radius: 5px !important;
}

/* 修改滑块手柄样式 */
.widget-slider .ui-slider-handle {
    width: 20px !important;               /* 增加手柄宽度 */
    height: 20px !important;              /* 增加手柄高度 */
    background: #2185d0 !important;       /* 蓝色手柄 */
    border: 2px solid white !important;
    border-radius: 50% !important;
    box-shadow: 0 0 5px rgba(0,0,0,0.2) !important;
}

/* 修改标签样式 */
.widget-label {
    min-width: 100px !important;
    max-width: none !important;
    width: auto !important;
    font-weight: bold !important;
}

/* 修改数值显示样式 */
.widget-readout {
    min-width: 60px !important;
    padding-left: 5px !important;
    font-weight: bold !important;
}

</style>
"""

display(HTML(custom_css))

# Display title
display(HTML('<div class="title">GMA News AI Assisted Research Tool</div>'))

# Create UI elements
date_input = widgets.DatePicker(description='Date:')
topic_input = widgets.Text(description='Topic:')
url_input = widgets.Text(description='URL:')

# Create new buttons with consistent size
run_button = widgets.Button(description='Run', layout=widgets.Layout(width='150px', height='40px'))
clear_button = widgets.Button(description='Clear', layout=widgets.Layout(width='150px', height='40px'))
save_button = widgets.Button(description='Save', layout=widgets.Layout(width='150px', height='40px'))
export_button = widgets.Button(description='Export', layout=widgets.Layout(width='150px', height='40px'))

output_tabs = widgets.Tab()
# Adjust output tabs layout
output_tabs.layout.height = '1000px'  # Increase this value as needed
output_tabs.layout.width = '100%'  # Keep width at 100%
# Create attribute output fields with bold labels and larger text areas
attribute_outputs = {}
for attr in attributes:

    # Create text area for the prompt
    prompt_area = widgets.Textarea(
        value='',
        placeholder='LLM prompt will appear here',
        disabled=True,  # Make it read-only
        layout=widgets.Layout(width='90%', height='200px')
    )
    ai_output_area = widgets.Textarea(
        value='',
        placeholder='Output will appear here',
        disabled=False,
        layout=widgets.Layout(width='100%', height='200px')
    )
    manual_output= widgets.Textarea(
        placeholder='Enter Manually created News Attributes',
        layout=widgets.Layout(width='100%', height='100px')
    )
    feedback = widgets.Textarea(
        placeholder='Enter feedback here',
        layout=widgets.Layout(width='100%', height='100px')
    )
    
    accuracy = widgets.FloatSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Accuracy (%):',  # 完整显示标签
    description_width='100px',    # 增加标签宽度
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'},  # 允许标签完整显示
    layout=widgets.Layout(
        width='80%',              # 设置整体宽度
        height='50px'             # 增加高度使滑块更粗
    )
    )
    attribute_outputs[attr] = {
        'prompt': prompt_area,
        'ai_output': ai_output_area,
        'manual_output':manual_output,
        'feedback': feedback,
        'accuracy': accuracy
    }

output_tabs.children = [
    widgets.VBox([
        
        widgets.Label('LLM Prompt:'),
        attribute_outputs[attr]['prompt'],
        widgets.Label('AI Generated News Attributes:'),
        attribute_outputs[attr]['ai_output'],
        widgets.Label('Expert Generated Attributes:'),
        attribute_outputs[attr]['manual_output'],
        widgets.Label('Feedback:'),
        attribute_outputs[attr]['feedback'],
        attribute_outputs[attr]['accuracy']
    ])
    for attr in attributes
]

for i, attr in enumerate(attributes):
    # short_title = attr[:35] + '...' if len(attr) > 35 else attr
    output_tabs.set_title(i, attr)
    output_tabs.children[i].layout.title = attr  # This sets the tooltip
    output_tabs.children[i].layout.width = '100%'

log_output = widgets.Output()

def clear_inputs(b):
    topic_input.value = ""
    url_input.value = ""
    date_input.value = None
    display(HTML("<b>Inputs cleared.</b>"))
    # Clear previous output
    with log_output:
        clear_output()
    # Clear attribute outputs
    for attr in attributes:
        attribute_outputs[attr]['prompt'].value = ''
        attribute_outputs[attr]['ai_output'].value = ''
        attribute_outputs[attr]['manual_output'].value = ''
        attribute_outputs[attr]['feedback'].value = ''
        attribute_outputs[attr]['accuracy'].value = 0

clear_button.on_click(clear_inputs)

def on_button_clicked(b):
    with log_output:
        clear_output()  # 清除之前的输出
        
        try:
            # 获取输入值
            date = date_input.value
            topic = topic_input.value
            url = url_input.value
            
            # 输入验证
            if not topic:
                display(HTML("<div style='color: red;'><b>错误:</b> 请输入主题</div>"))
                return
            if not url:
                display(HTML("<div style='color: red;'><b>错误:</b> 请输入URL</div>"))
                return
                
            # 显示开始处理的信息
            display(HTML("<div style='color: blue;'><b>开始处理...</b></div>"))
            
            # 清除之前的输出
            for attr in attributes:
                attribute_outputs[attr]['ai_output'].value = ''
            
            # 调用处理函数
            process_url(date, topic, url, attribute_outputs)
            
        except Exception as e:
            display(HTML(f"<div style='color: red;'><b>处理过程中出错:</b> {str(e)}</div>"))

# Connect the button click event to the function
run_button.on_click(on_button_clicked)

# Define response_col_index globally
header_row = output_sheet[1]
response_column = 'AI Generated Answer'
response_col_index = next((i for i, cell in enumerate(header_row, 1) if cell.value == response_column), None)
if not response_col_index:
    raise ValueError(f"Required column {response_column} not found in Output sheet.")

# Add new columns to the output sheet
output_sheet.cell(row=1, column=response_col_index + 1, value='manual_output')
output_sheet.cell(row=1, column=response_col_index + 1, value='Feedback')
output_sheet.cell(row=1, column=response_col_index + 2, value='Accuracy (%)')

def save_changes(b):
    with log_output:
        clear_output()
    for attr, outputs in attribute_outputs.items():
        row_index = attributes.index(attr) + 2
        output_sheet.cell(row=row_index, column=response_col_index, value=outputs['prompt'].value)
        output_sheet.cell(row=row_index, column=response_col_index, value=outputs['ai_output'].value)
        output_sheet.cell(row=row_index, column=response_col_index, value=outputs['manual_output'].value)
        output_sheet.cell(row=row_index, column=response_col_index + 1, value=outputs['feedback'].value)
        output_sheet.cell(row=row_index, column=response_col_index + 2, value=outputs['accuracy'].value)
    wb.save(file_path)
    display(HTML("<b>Changes saved successfully!</b>"))

def export_to_excel(b):
    with log_output:
        clear_output()
    export_data = []
    for attr, outputs in attribute_outputs.items():
        export_data.append([
            attr,
            outputs['prompt'].value,
            outputs['ai_output'].value,
            outputs['manual_output'].value,
            outputs['feedback'].value,
            outputs['accuracy'].value
        ])
    df = pd.DataFrame(export_data, columns=['Attribute', 'Content', 'AI Output','Manual Output','Feedback', 'Accuracy (%)'])
    wb_export = Workbook()
    ws = wb_export.active
    ws.title = "Exported"

    # Write headers
    headers = ['Attribute', 'LLM Prompt', 'AI Generated','Expert Created','Feedback', 'Accuracy (%)']
    for col, header in enumerate(headers, start=1):
        ws.cell(row=1, column=col, value=header).font = Font(bold=True)

    # Write data
    for r, row in enumerate(df.values, start=2):
        for c, value in enumerate(row, start=1):
            ws.cell(row=r, column=c, value=value)

    # Adjust column widths
    ws.column_dimensions['A'].width = 30
    ws.column_dimensions['B'].width = 80
    ws.column_dimensions['C'].width = 50
    ws.column_dimensions['D'].width = 15

    # Create a root window and hide it
    root = tk.Tk()
    root.withdraw()

    # Ask user for export location and filename
    export_path = filedialog.asksaveasfilename(
        defaultextension=".xlsx",
        filetypes=[("Excel files", "*.xlsx")],
        title="Save Excel File"
    )

    if not export_path:
        # User cancelled the dialog
        display(HTML("<b>Export cancelled.</b>"))
        return

    # Save the workbook
    wb_export.save(export_path)
    display(HTML(f"<b>Data exported successfully to:</b> {export_path}"))

# Connect buttons to functions
save_button.on_click(save_changes)
export_button.on_click(export_to_excel)

# Update the UI display to include all buttons next to each other
display(widgets.VBox([
    widgets.HBox([date_input], layout=widgets.Layout(justify_content='flex-start')),
    widgets.HBox([topic_input], layout=widgets.Layout(justify_content='flex-start')),
    widgets.HBox([url_input], layout=widgets.Layout(justify_content='flex-start')),
    widgets.HTML('<div class="banner">'),
    widgets.HBox([run_button, clear_button, save_button, export_button], layout=widgets.Layout(justify_content='flex-start')),
    widgets.HTML('</div>'),
    widgets.HTML('<div class="note">Note: Topic and URL are mandatory. Please describe the news in the topic field and provide the URL. Date is an optional input.</div>')
]))

display(output_tabs)
display(log_output)


Output()